Rayudu- The Legal Advisor.

## Installing the necessary libraries

In [1]:
# Install Hugging Face Transformers, Datasets, and Accelerate
!pip install transformers datasets accelerate

# Install PyTorch (should be pre-installed, but just in case)
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

## model training

In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
from sklearn.model_selection import train_test_split

# Load the CSV file
csv_file = 'legal_qa.csv'
df = pd.read_csv(csv_file)

# Prepare the dataset
df['text'] = df['Question'] + " " + df['Answer']
train_df, test_df = train_test_split(df[['text']], test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Add a padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the data
def tokenize_function(examples):
    encodings = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

    encodings['labels'] = encodings['input_ids'].copy()
    return encodings

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Ensuring the datasets are formatted correctly
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=150,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=200,
    save_total_limit=5,
    learning_rate=3e-5,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Early stopping if no improvement for 3 evals
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
200,0.226100,0.445231
400,0.119600,0.427449
600,0.077500,0.455105
800,0.056300,0.471978
1000,0.053400,0.488477


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1000, training_loss=0.2509671633243561, metrics={'train_runtime': 133.096, 'train_samples_per_second': 220.893, 'train_steps_per_second': 55.223, 'total_flos': 261292032000000.0, 'train_loss': 0.2509671633243561, 'epoch': 20.408163265306122})

In [4]:
# Load the best model checkpoint manually
best_checkpoint = "./results/checkpoint-1000"
model = AutoModelForCausalLM.from_pretrained(best_checkpoint)

# Save this best model
model.save_pretrained("./fine-tuned-gpt2-best")
tokenizer.save_pretrained("./fine-tuned-gpt2-best")

('./fine-tuned-gpt2-best/tokenizer_config.json',
 './fine-tuned-gpt2-best/special_tokens_map.json',
 './fine-tuned-gpt2-best/vocab.json',
 './fine-tuned-gpt2-best/merges.txt',
 './fine-tuned-gpt2-best/added_tokens.json',
 './fine-tuned-gpt2-best/tokenizer.json')

In [5]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model and tokenizer
model_path = "./fine-tuned-gpt2-best"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Function to generate a response
def generate_response(prompt, max_length=150, num_beams=5, early_stopping=True):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_return_sequences=1,
        num_beams=num_beams,  # Beam search for better quality
        early_stopping=early_stopping,
        pad_token_id=tokenizer.eos_token_id  # Set pad token id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test the chatbot with some example questions
example_questions = [
    " What’s sexual act"

]

# Generate and print responses for the example questions
for question in example_questions:
    print(f"Question: {question}")
    print(f"Response: {generate_response(question)}")
    print("-" * 50)



    # Save this best model
model.save_pretrained("./fine-tuned-gpt2-best")
tokenizer.save_pretrained("./fine-tuned-gpt2-best")

Question:  What’s sexual act
Response:  What’s sexual act’? Sexual act refers to the physical act or unlawful omission that constitutes a crime.
--------------------------------------------------


('./fine-tuned-gpt2-best/tokenizer_config.json',
 './fine-tuned-gpt2-best/special_tokens_map.json',
 './fine-tuned-gpt2-best/vocab.json',
 './fine-tuned-gpt2-best/merges.txt',
 './fine-tuned-gpt2-best/added_tokens.json',
 './fine-tuned-gpt2-best/tokenizer.json')

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [7]:
# Load the fine-tuned model and tokenizer
model_path = "./fine-tuned-gpt2-best"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)


In [8]:
# Function to generate a response
def generate_response(prompt, max_length=150, num_beams=5, early_stopping=True):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
                inputs["input_ids"],
                        attention_mask=inputs["attention_mask"],
                                max_length=max_length,
                                        num_return_sequences=1,
                                                num_beams=num_beams,  # Beam search for better quality
                                                        early_stopping=early_stopping,
                                                                pad_token_id=tokenizer.eos_token_id  # Set pad token id
                                                                    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [9]:
# Interactive chatbot
def chat():
    print("Chatbot is ready! Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
           print("Chatbot: Goodbye!")
           break
        response = generate_response(user_input)
        print(f"Chatbot: {response}")
        user_satisfaction = input("Are you satisfied with the answer? (ok/continue): ").strip().lower()
        if user_satisfaction == 'ok':
           print("Chatbot: Glad I could help! Goodbye!")
           break

if __name__ == "__main__":
   chat()

Chatbot is ready! Type 'quit' to exit.


KeyboardInterrupt: Interrupted by user